### Import Libraries

In [ ]:
import uuid
import time
import requests
from openai import OpenAI
from runwayml import RunwayML
from PIL import Image
from IPython.display import display, Video
from dotenv import load_dotenv
load_dotenv()

# The following env variables must be set:
# OPENAI_API_KEY
# RUNWAYML_API_SECRET

### Define Prompts

In [ ]:
img_prompt = "A cat in a dapper 1940s attire, sipping a frosty glass of beer at a bustling farm-to-table restaurant"
video_prompt = "The cat is fidgeting anxiously in his chair"

### Generate Static Image Using DALL-E

In [ ]:

client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt=img_prompt,
  size="1024x1792",
  quality="standard",
)

image_url = response.data[0].url
image = requests.get(image_url)
image_name = f"{uuid.uuid4()}.png"

with open(f"images/{image_name}", "wb") as file:
    file.write(image.content)

img = Image.open(f"images/{image_name}")
img.thumbnail((768, 768))
display(img)

### Generate Video From Image Using Runway

In [ ]:
runway_client = RunwayML()

task = runway_client.image_to_video.create(
  model='gen3a_turbo',
  duration=5,
  ratio='768:1280',
  prompt_image=image_url,
  prompt_text=video_prompt,
)
print(task.id)

while True:
    status_task = runway_client.tasks.retrieve(id=task.id)
    if status_task.status != 'RUNNING' and status_task.status != 'PENDING':
        break
    time.sleep(1)

if status_task.status == 'SUCCEEDED':
    video_url = status_task.output[0]
    print(video_url)
    video = requests.get(video_url)
    video_name = f"{image_name}.mp4"
    with open(f"images/{video_name}", "wb") as file:
        file.write(video.content)
    display(Video(f"images/{video_name}"))
else:
    print("Video generation failed")